In [ ]:
import whisper
import torch
import numpy as np
import sounddevice as sd
import os
from sentence_transformers import SentenceTransformer
import pandas as pd
from IPython.display import Audio, display
from huggingface_hub import login

In [ ]:
# Load the sheet named "Merged" from the Excel file
df = pd.read_excel("Hand_Signs.xlsx", sheet_name="Merged", header=None)

# Rename the columns
df.columns = ["Text", "Pose_Sequence"]

In [3]:
# Normalize the "Poses sequences" column to always be a list of integers
def parse_pose_sequence(val):
    if isinstance(val, list):
        return val
    if pd.isna(val):  # Check for NaN
        return []
    if isinstance(val, (int, float)):
        return [int(val)]
    if isinstance(val, str):
        # Filter out empty strings before converting
        return [int(x.strip()) for x in val.split(",") if x.strip().isdigit()]
    return []

df["Pose_Sequence"] = df["Pose_Sequence"].apply(parse_pose_sequence)

### Add embeddings column

In [ ]:
# Login to hugging face 
login(token="hugging_face_token")

In [ ]:
# Load the model (downloads on first run)
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
print("Embedding model loaded successfully!")

def text_to_embeddings(text):
    """Converts English text to embeddings using SentenceTransformer."""
    embeddings = embedding_model.encode(text, convert_to_tensor=True)
    return embeddings.numpy()  # Return as NumPy array for easier handling


Embedding model loaded successfully!


In [6]:
# Compute embeddings and insert them in the middle
df.insert(loc=1, column="Embeddings", value=df["Text"].apply(text_to_embeddings))
df["Embeddings"] = df["Embeddings"].apply(lambda x: x.tolist() if hasattr(x, 'tolist') else x)
df.head()


,Text,Embeddings,Pose_Sequence
0,Show me 1,"[-0.02809363603591919, 0.019465146586298943, -...",[1]
1,Show me 2,"[-0.0219540074467659, 0.036779001355171204, -0...",[2]
2,Show me 3,"[-0.01712719351053238, -0.007692048326134682, ...",[3]
3,Show me 4,"[-0.011031582951545715, 0.0035835669841617346,...",[4]
4,Show me 5,"[-0.07056542485952377, -0.027202408760786057, ...",[5]


In [7]:
# Save df to CSV
df.to_csv("preprocessed_dataset.csv", index=False)

### Read preprocessed csv


In [8]:
import ast  # for safely evaluating string representations of lists

# Define parsing function for lists
def parse_list(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return []

# Read the CSV and parse list columns
df_loaded = pd.read_csv("preprocessed_dataset.csv", converters={
    "Embeddings": parse_list,
    "Pose_Sequence": parse_list
})


In [9]:
print(df_loaded.head())
print(df_loaded.dtypes)


        Text                                         Embeddings Pose_Sequence
0  Show me 1  [-0.02809363603591919, 0.019465146586298943, -...           [1]
1  Show me 2  [-0.0219540074467659, 0.036779001355171204, -0...           [2]
2  Show me 3  [-0.01712719351053238, -0.007692048326134682, ...           [3]
3  Show me 4  [-0.011031582951545715, 0.0035835669841617346,...           [4]
4  Show me 5  [-0.07056542485952377, -0.027202408760786057, ...           [5]
Text             object
Embeddings       object
Pose_Sequence    object
dtype: object
